In [1]:
%load_ext autoreload
%autoreload 2

In [37]:
import random
import time
from utils import *
from models import *
from attention import *

import torch
import torch.optim as optim
from torch.utils import data
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Download the file
get_file()
path_to_file = 'spa-eng/spa.txt'

======== THE DATASET IS READY ========


In [4]:
# Try experimenting with the size of that dataset
num_examples = 10000#118960
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

The number of line in the dataset is 118964


In [5]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

9000 9000 1000 1000


In [6]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
20 ----> se
1688 ----> abrazaron
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
20 ----> they
637 ----> hugged
3 ----> .
2 ----> <end>


In [7]:
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)// BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

In [8]:
# transform to torch tensor
tensor_x = torch.Tensor(input_tensor_train).long() 
tensor_y = torch.Tensor(target_tensor_train).long()
# create your datset
my_dataset = data.TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = data.DataLoader(my_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True,
                        num_workers=4)   

In [9]:
example_input_batch, example_target_batch = next(iter(my_dataloader))

In [10]:
example_input_batch.shape

torch.Size([64, 12])

In [11]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) torch.Size([64, 12, 1024])
Encoder Hidden state shape: (batch size, units) torch.Size([1, 64, 1024])


In [12]:
attention_layer = BahdanauAttention(10, 1024)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 12, 1])


In [13]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024)

sample_decoder_output, _, _ = decoder(torch.randint(1, 20, (BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) torch.Size([64, 2306])


In [14]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE).to(device)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [15]:
# Train the model
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(my_dataloader):
        inp, targ = inp.to(device), targ.to(device)
        batch_loss = train_step(inp, targ, encoder, decoder,
                                encoder_optimizer, decoder_optimizer,
                                criterion, device, BATCH_SIZE, targ_lang)
        
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
            
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        
        pass

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 6.7608
Epoch 1 Batch 100 Loss 2.6164
Epoch 1 Loss 2.6480
Time taken for 1 epoch 9.188030481338501 sec



In [16]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> trata de averiguarlo . <end>
Predicted translation: i need . <end> 


In [17]:
translate(u'¿ todavia estan en casa ?', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> ¿ todavia estan en casa ? <end>
Predicted translation: how do it ? <end> 


In [18]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

Input: <start> esta es mi vida . <end>
Predicted translation: this is a . <end> 


In [113]:
translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

	
toto 1
1.3507747650146484
1.5893034934997559
2.876343250274658
	
toto 1
1.4791193008422852
1.7473859786987305
2.9865612983703613
1.4779396057128906
1.7460575103759766
2.9872875213623047
1.4780220985412598
1.746159553527832
2.987255573272705
	
toto 3
1.4780211448669434
1.746157169342041
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
1.4780220985412598
1.7461585998535156
2.9872565269470215
Input: <start> hace mucho frio aqui . <end>
Predicted translation: 


In [29]:
from queue import Queue, PriorityQueue, LifoQueue

In [30]:
nodes = LifoQueue()
#nodes = PriorityQueue()

In [35]:
nodes.put(("c", 1))
nodes.put(("b", 2))
nodes.put(("a", 3))

In [36]:
nodes.get()

('a', 3)

In [74]:
m = BeamTreeMates(None)

In [75]:
BeamTreeMates.nbr

1

In [76]:
q = BeamTreeMates(None)

In [78]:
q.nbr

2

# FIN